In [1]:
# Loading latest version of pdf_parser
# auto reloads the module if it has been changed

from pdf_parser import PdfParser

parser = PdfParser()

/Users/lesleywang/anaconda3/envs/surya/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing Docment Parser......
Loading Surya models......
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Initializing Tesseract......


In [70]:
pdf_bytes = open('pic2.pdf', 'rb').read()

df = parser.parse_pdf(pdf_bytes)
df.to_excel('output_pic2.xlsx', index=False)

Finding reading order: 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


In [71]:
table_df = df[df['label'] == 'Table']
table_df

,page_idx,position,bbox,label,text
2,0,2,"(152, 252, 1076, 1289)",Table,SLALOMeHt OF PInahtial PUSIIOH\n\nASSETS Notes...


In [15]:
def extract_table_data_from_excel(file_path):
    """
    Extract only the rows labeled as 'Table' from an Excel file and return the data in matrix format
    containing page_idx, position, bbox, label, and text.
    """
    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Filter for rows labeled as 'Table'
    table_df = df[df['label'] == 'Table']
    
    # Select necessary columns
    table_matrix = table_df[['page_idx', 'position', 'bbox', 'label']]
    
    return table_matrix

# Call the function with the uploaded file path
file_path = 'output.xlsx'
filtered_table_matrix = extract_table_data_from_excel(file_path)


In [16]:
filtered_table_matrix

,page_idx,position,bbox,label
2,1,1,"(176, 141, 1074, 710)",Table
80,7,2,"(96, 198, 1101, 755)",Table
83,8,1,"(89, 176, 1121, 1331)",Table
88,9,1,"(90, 174, 1204, 621)",Table
91,10,1,"(89, 146, 1126, 1051)",Table
121,12,11,"(153, 1078, 635, 1218)",Table
130,13,2,"(113, 194, 1122, 1500)",Table
135,14,3,"(147, 209, 1100, 425)",Table
137,14,5,"(135, 492, 1115, 917)",Table
141,14,9,"(147, 1041, 1100, 1237)",Table


In [18]:
from PIL import Image

def objects_to_crops_from_df(pdf_pages, tokens, table_df, padding=10):
    """
    Process the bounding boxes from the dataframe into cropped table images 
    and cropped tokens for selected pages in the PDF.
    
    Args:
    - pdf_pages: list of images, where each image is a page of the PDF.
    - tokens: list of tokens to adjust.
    - table_df: dataframe containing page_idx, position, bbox, label for tables.
    - padding: int, padding around the bbox.
    
    Returns:
    - A list of cropped table images and their respective tokens.
    """

    table_crops = []

    # Iterate through each row in the DataFrame
    for _, row in table_df.iterrows():
        page_idx = row['page_idx']
        bbox = eval(row['bbox'])  # Convert the bbox string into a list of integers
        label = row['label']

        # Get the corresponding page image
        if page_idx >= len(pdf_pages):
            continue  # Skip if the page index is out of range

        img = pdf_pages[page_idx]

        # Apply padding to the bounding box
        bbox = [bbox[0]-padding, bbox[1]-padding, bbox[2]+padding, bbox[3]+padding]

        # Crop the image
        cropped_img = img.crop(bbox)

        # Filter tokens within the bounding box
        table_tokens = [token for token in tokens if iob(token['bbox'], bbox) >= 0.5]
        for token in table_tokens:
            token['bbox'] = [token['bbox'][0]-bbox[0],
                             token['bbox'][1]-bbox[1],
                             token['bbox'][2]-bbox[0],
                             token['bbox'][3]-bbox[1]]

        # Rotate if the table is labeled as 'table rotated'
        if label == 'table rotated':
            cropped_img = cropped_img.rotate(270, expand=True)
            for token in table_tokens:
                token_bbox = token['bbox']
                token['bbox'] = [cropped_img.size[0]-token_bbox[3]-1,
                                 token_bbox[0],
                                 cropped_img.size[0]-token_bbox[1]-1,
                                 token_bbox[2]]

        cropped_table = {'image': cropped_img, 'tokens': table_tokens}
        table_crops.append(cropped_table)

    return table_crops


# Example usage:

# Assuming `image_list` is a list of PIL images representing pages of a PDF
# and `table_df` is the DataFrame with `page_idx`, `position`, `bbox`, and `label`
image_list = [Image.open(f"11_AuditedStatements.pdf")]  # Load your PDF pages here
tokens = []  # Replace with your list of tokens

# Crop tables
cropped_tables = objects_to_crops_from_df(image_list, tokens, table_df)

# Access the first cropped table image
if cropped_tables:
    cropped_table_image = cropped_tables[0]['image'].convert("RGB")
    cropped_table_image.show()


UnidentifiedImageError: cannot identify image file '/Users/lesleywang/Desktop/demo 1001/11_AuditedStatements.pdf'

In [20]:
pip install pdfplumber

  Using cached cryptography-43.0.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (5.4 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 2.9 MB/s eta 0:00:0000:0100:01
Using cached cryptography-43.0.1-cp39-abi3-macosx_10_9_universal2.whl (6.2 MB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
Note: you may need to restart the kernel to use updated packages.


In [27]:
import pdfplumber
from PIL import Image

def crop_pdf_sections_to_png(pdf_path, sections, output_dir):
    """
    Crop specified sections from a PDF and save them as PNG images.

    Parameters:
    - pdf_path: str, path to the PDF file.
    - sections: list of dicts, each containing 'page_idx', 'bbox', and 'label'.
    - output_dir: str, directory to save the cropped PNG images.
    """
    with pdfplumber.open(pdf_path) as pdf:
        for section in sections:
            #convert page_idx to int
            page_idx = int(section['page_idx'])
             # Convert the bbox from string to tuple using ast.literal_eval
            bbox = ast.literal_eval(row['bbox'])
            #bbox = section['bbox']  # (x0, y0, x1, y1)
            label = section['label']

            # Extract the specified page
            page = pdf.pages[page_idx - 1]  # Adjusting for zero-based index

            # Crop the page to the specified bounding box
            cropped_image = page.crop(bbox).to_image()

            # Save the cropped image as a PNG file
            output_path = f"{output_dir}/{label}_page{page_idx}.png"
            cropped_image.save(output_path, 'PNG')
            print(f"Saved cropped section as {output_path}")


In [28]:
# Convert the data into a DataFrame
df = pd.DataFrame(filtered_table_matrix)

# Specify the PDF file and output directory
pdf_path = '11_AuditedStatements.pdf'  # Path to your PDF
output_dir = "output"

# Call the function to crop and save PNGs
crop_pdf_sections_to_png(pdf_path, df, output_dir)

TypeError: string indices must be integers

In [33]:
import pdfplumber
from PIL import Image
import ast
import os

def crop_pdf_sections_to_png(pdf_path, sections, output_dir):
    """
    Crop specified sections from a PDF and save them as PNG images.

    Parameters:
    - pdf_path: str, path to the PDF file.
    - sections: list of dicts, each containing 'page_idx', 'bbox', and 'label'.
    - output_dir: str, directory to save the cropped PNG images.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with pdfplumber.open(pdf_path) as pdf:
        for section in sections:
            # Convert page_idx to int
            page_idx = int(section['page_idx'])
            
            # Convert the bbox from string to tuple using ast.literal_eval
            bbox = ast.literal_eval(section['bbox'])
            label = section['label']

            # Extract the specified page
            page = pdf.pages[page_idx - 1]  # Adjusting for zero-based index

            # Crop the page to the specified bounding box
            cropped_image = page.within_bbox(bbox).to_image()

            # Save the cropped image as a PNG file
            output_path = f"{output_dir}/{label}_page{page_idx}.png"
            cropped_image.save(output_path, 'PNG')
            print(f"Saved cropped section as {output_path}")

# Convert DataFrame rows to a list of dictionaries
def convert_df_to_dict_list(df):
    """
    Convert a Pandas DataFrame to a list of dictionaries.
    
    Each row is converted to a dictionary containing 'page_idx', 'bbox', and 'label'.
    """
    return df.to_dict(orient='records')

# Example usage with a DataFrame
import pandas as pd


In [34]:

df = pd.DataFrame(filtered_table_matrix)

# Convert the DataFrame to a list of dictionaries
sections = convert_df_to_dict_list(df)

pdf_path = '11_AuditedStatements.pdf'
output_dir = "output"

# Call the function to crop and save PNGs
crop_pdf_sections_to_png(pdf_path, sections, output_dir)


ValueError: Bounding box (176, 141, 1074, 710) is not fully within parent page bounding box (0, 0.0, 595.2, 841.8)

In [37]:
import pdfplumber
from PIL import Image
import ast
import os

def adjust_bbox(bbox, page_width, page_height):
    """
    Adjust the bounding box to ensure it fits within the page dimensions.
    """
    x0, y0, x1, y1 = bbox
    # Ensure the bbox coordinates are within the page size
    x0 = max(0, min(x0, page_width))
    y0 = max(0, min(y0, page_height))
    x1 = max(0, min(x1, page_width))
    y1 = max(0, min(y1, page_height))
    return (x0, y0, x1, y1)

def crop_pdf_sections_to_png(pdf_path, sections, output_dir):
    """
    Crop specified sections from a PDF and save them as PNG images.

    Parameters:
    - pdf_path: str, path to the PDF file.
    - sections: list of dicts, each containing 'page_idx', 'bbox', and 'label'.
    - output_dir: str, directory to save the cropped PNG images.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with pdfplumber.open(pdf_path) as pdf:
        for section in sections:
            # Convert page_idx to int
            page_idx = int(section['page_idx'])
            
            # Convert the bbox from string to tuple using ast.literal_eval
            bbox = ast.literal_eval(section['bbox'])
            label = section['label']

            # Extract the specified page
            page = pdf.pages[page_idx - 1]  # Adjust for zero-based index

            # Get page dimensions
            page_width = page.width
            page_height = page.height

            # Adjust the bounding box to fit within the page
            adjusted_bbox = adjust_bbox(bbox, page_width, page_height)

            # Crop the page to the adjusted bounding box
            cropped_image = page.within_bbox(adjusted_bbox).to_image()

            # Save the cropped image as a PNG file
            output_path = f"{output_dir}/{label}_page{page_idx}.png"
            cropped_image.save(output_path, 'PNG')
            print(f"Saved cropped section as {output_path}")

# Convert DataFrame rows to a list of dictionaries
def convert_df_to_dict_list(df):
    """
    Convert a Pandas DataFrame to a list of dictionaries.
    
    Each row is converted to a dictionary containing 'page_idx', 'bbox', and 'label'.
    """
    return df.to_dict(orient='records')

# Example usage with a DataFrame
import pandas as pd


df = pd.DataFrame(filtered_table_matrix)

# Convert the DataFrame to a list of dictionaries
sections = convert_df_to_dict_list(df)

pdf_path = '11_AuditedStatements.pdf'
output_dir = "output"

# Call the function to crop and save PNGs
crop_pdf_sections_to_png(pdf_path, sections, output_dir)


Saved cropped section as output/Table_page1.png
Saved cropped section as output/Table_page7.png
Saved cropped section as output/Table_page8.png
Saved cropped section as output/Table_page9.png
Saved cropped section as output/Table_page10.png


ValueError: Bounding box (153, 841.8, 595.2, 841.8) has an area of zero.

In [50]:
import pdfplumber
from PIL import Image
import ast
import os


def adjust_bbox(bbox, page_width, page_height, padding=10):
    x0, y0, x1, y1 = bbox
    # Apply padding and ensure the bbox coordinates are within the page size
    x0 = max(0, x0 - padding)
    y0 = max(0, y0 - padding)
    x1 = min(page_width, x1 + padding)
    y1 = min(page_height, y1 + padding)
    return (x0, y0, x1, y1)

def is_valid_bbox(bbox):
    """
    Check if the bounding box defines a non-zero area.
    """
    x0, y0, x1, y1 = bbox
    return (x1 - x0) > 0 and (y1 - y0) > 0

def crop_pdf_sections_to_png(pdf_path, sections, output_dir):
    """
    Crop specified sections from a PDF and save them as PNG images.

    Parameters:
    - pdf_path: str, path to the PDF file.
    - sections: list of dicts, each containing 'page_idx', 'bbox', and 'label'.
    - output_dir: str, directory to save the cropped PNG images.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with pdfplumber.open(pdf_path) as pdf:
        for section in sections:
            # Convert page_idx to int
            page_idx = int(section['page_idx'])
            
            # Convert the bbox from string to tuple using ast.literal_eval
            bbox = ast.literal_eval(section['bbox'])
            label = section['label']

            # Extract the specified page
            page = pdf.pages[page_idx - 1]  # Adjusting for zero-based index

            # Get page dimensions
            page_width = page.width
            page_height = page.height

            # Adjust the bounding box to fit within the page
            adjusted_bbox = adjust_bbox(bbox, page_width, page_height)

            # Check if the bounding box has a valid area before cropping
            if not is_valid_bbox(adjusted_bbox):
                print(f"Skipping invalid bounding box {adjusted_bbox} on page {page_idx}")
                continue

            # Crop the page to the adjusted bounding box
            cropped_image = page.within_bbox(adjusted_bbox).to_image()

            # Save the cropped image as a PNG file
            output_path = f"{output_dir}/{label}_page{page_idx}.png"
            cropped_image.save(output_path, 'PNG')
            print(f"Saved cropped section as {output_path}")

# Convert DataFrame rows to a list of dictionaries
def convert_df_to_dict_list(df):
    """
    Convert a Pandas DataFrame to a list of dictionaries.
    
    Each row is converted to a dictionary containing 'page_idx', 'bbox', and 'label'.
    """
    return df.to_dict(orient='records')


# Example usage with a DataFrame
import pandas as pd


df = pd.DataFrame(filtered_table_matrix)

# Convert the DataFrame to a list of dictionaries
sections = convert_df_to_dict_list(df)

pdf_path = '11_AuditedStatements.pdf'
output_dir = "output_new"

# Call the function to crop and save PNGs
crop_pdf_sections_to_png(pdf_path, sections, output_dir)


Saved cropped section as output_new/Table_page1.png
Saved cropped section as output_new/Table_page7.png
Saved cropped section as output_new/Table_page8.png
Saved cropped section as output_new/Table_page9.png
Saved cropped section as output_new/Table_page10.png
Skipping invalid bounding box (143, 1068, 595.2, 841.8) on page 12
Saved cropped section as output_new/Table_page13.png
Saved cropped section as output_new/Table_page14.png
Saved cropped section as output_new/Table_page14.png
Skipping invalid bounding box (137, 1031, 595.2, 841.8) on page 14
Skipping invalid bounding box (135, 1298, 595.2, 841.8) on page 14
Saved cropped section as output_new/Table_page15.png
Saved cropped section as output_new/Table_page16.png
Skipping invalid bounding box (84, 849, 841.8, 595.2) on page 16
Saved cropped section as output_new/Table_page17.png


In [59]:
import pdfplumber
from PIL import Image
import ast
import os

def is_bbox_within_page(bbox, page_width, page_height):
    """
    Check if the bounding box is within the page dimensions.
    """
    x0, y0, x1, y1 = bbox
    return (0 <= x0 < page_width and 0 <= y0 < page_height and
            0 <= x1 <= page_width and 0 <= y1 <= page_height)

def adjust_bbox(bbox, page_width, page_height, padding=0):
    x0, y0, x1, y1 = bbox
    # Apply padding and ensure the bbox coordinates are within the page size
    #x0 = max(0, x0 - padding)
    #y0 = max(0, y0 - padding)
    #x1 = min(page_width, x1 - padding)
    #y1 = min(page_height, y1 - padding)
    x0 = x0 - padding
    y0 = y0 - padding
    x1 = x1 + padding
    y1 = y1 + padding
    #y0 = max(0, y0 - padding)
    #x1 = min(page_width, x1 + padding)
    #y1 = min(page_height, y1 + padding)
    return (x0, y0, x1, y1)

def is_valid_bbox(bbox):
    """
    Check if the bounding box defines a non-zero area.
    """
    x0, y0, x1, y1 = bbox
    return (x1 - x0) > 0 and (y1 - y0) > 0

def crop_pdf_sections_to_png(pdf_path, sections, output_dir, padding=100):
    """
    Crop specified sections from a PDF and save them as PNG images.

    Parameters:
    - pdf_path: str, path to the PDF file.
    - sections: list of dicts, each containing 'page_idx', 'bbox', and 'label'.
    - output_dir: str, directory to save the cropped PNG images.
    - padding: int, padding to apply to all bounding boxes (default 10).
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with pdfplumber.open(pdf_path) as pdf:
        for section in sections:
            # Convert page_idx to int
            page_idx = int(section['page_idx'])
            
            # Convert the bbox from string to tuple using ast.literal_eval
            bbox = ast.literal_eval(section['bbox'])
            label = section['label']

            # Extract the specified page
            page = pdf.pages[page_idx - 1]  # Adjusting for zero-based index

            # Get page dimensions
            page_width = page.width
            page_height = page.height

            # Adjust the bounding box to fit within the page and apply padding
            adjusted_bbox = adjust_bbox(bbox, page_width, page_height, padding=padding)

            # Check if the bounding box is within the page
            if not is_bbox_within_page(adjusted_bbox, page_width, page_height):
                print(f"Skipping section with bbox {adjusted_bbox} on page {page_idx} (out of bounds)")
                continue

            # Crop the page to the adjusted bounding box
            cropped_image = page.within_bbox(adjusted_bbox).to_image()

            # Save the cropped image as a PNG file
            output_path = f"{output_dir}/{label}_page{page_idx}.png"
            cropped_image.save(output_path, 'PNG')
            print(f"Saved cropped section as {output_path}")

# Convert DataFrame rows to a list of dictionaries
def convert_df_to_dict_list(df):
    """
    Convert a Pandas DataFrame to a list of dictionaries.
    
    Each row is converted to a dictionary containing 'page_idx', 'bbox', and 'label'.
    """
    return df.to_dict(orient='records')


# Example usage with a DataFrame
import pandas as pd


df = pd.DataFrame(filtered_table_matrix)

# Convert the DataFrame to a list of dictionaries
sections = convert_df_to_dict_list(df)

pdf_path = '11_AuditedStatements.pdf'
output_dir = "output_new"

# Call the function to crop and save PNGs
crop_pdf_sections_to_png(pdf_path, sections, output_dir)


Skipping section with bbox (76, 41, 1174, 810) on page 1 (out of bounds)
Skipping section with bbox (-4, 98, 1201, 855) on page 7 (out of bounds)
Skipping section with bbox (-11, 76, 1221, 1431) on page 8 (out of bounds)
Skipping section with bbox (-10, 74, 1304, 721) on page 9 (out of bounds)
Skipping section with bbox (-11, 46, 1226, 1151) on page 10 (out of bounds)
Skipping section with bbox (53, 978, 735, 1318) on page 12 (out of bounds)
Skipping section with bbox (13, 94, 1222, 1600) on page 13 (out of bounds)
Skipping section with bbox (47, 109, 1200, 525) on page 14 (out of bounds)
Skipping section with bbox (35, 392, 1215, 1017) on page 14 (out of bounds)
Skipping section with bbox (47, 941, 1200, 1337) on page 14 (out of bounds)
Skipping section with bbox (45, 1208, 1203, 1667) on page 14 (out of bounds)
Skipping section with bbox (7, 187, 1708, 1023) on page 15 (out of bounds)
Skipping section with bbox (-10, 127, 1165, 896) on page 16 (out of bounds)
Skipping section with bb

In [62]:
import pdfplumber

def get_page_boundaries(pdf_path, page_index):
    """
    Get the boundary (width and height) of a specific page in a PDF.
    
    Parameters:
    - pdf_path: str, path to the PDF file.
    - page_index: int, the index of the page to inspect (0-based index).
    
    Returns:
    - tuple: (width, height) of the page.
    """
    with pdfplumber.open(pdf_path) as pdf:
        # Get the specified page
        page = pdf.pages[page_index]
        
        # Get the width and height of the page
        page_width = page.width
        page_height = page.height
        
        return page_width, page_height

# Example usage:
pdf_path = '11_AuditedStatements.pdf'
page_index = 3  # First page (0-based index)

# Get the page boundaries
page_width, page_height = get_page_boundaries(pdf_path, page_index)
print(f"Page width: {page_width}, Page height: {page_height}")


Page width: 595.2, Page height: 841.8


In [67]:
def get_bboxes(layout_predictions):
    '''
    Get the bounding boxes from the layout predictions and ensure they fit within the page dimensions.
    '''
    bboxes = []
    
    for page_idx, page in enumerate(layout_predictions):
        temp = []
        page_width, page_height = page.width, page.height  # Assuming you can get these dimensions
        for block_idx, block in enumerate(page.bboxes):
            bbox = block.bbox
            # Check if bbox is within page boundaries
            x0, y0, x1, y1 = bbox
            if x0 >= 0 and y0 >= 0 and x1 <= page_width and y1 <= page_height:
                temp.append(bbox)
            else:
                print(f"Skipping invalid bbox {bbox} on page {page_idx} (out of bounds)")
        bboxes.append(temp)
        
    return bboxes

# Example usage:
pdf_path = 'pic2.pdf'
page_index = 3  # First page (0-based index)

# Get the page boundaries
page_height = get_bboxes(pdf_path)


AttributeError: 'str' object has no attribute 'width'

In [75]:
import pdfplumber

# Open the single-page PDF file
with pdfplumber.open("pic2.pdf") as pdf:
    # Since the PDF has only one page, we can directly access the first page
    page = pdf.pages[0]  # First (and only) page
    
    # Define the adjusted bounding box to fit within the page's size (595, 842)
    table_bbox = (152, 252, 595, 842)  # Adjust x1 and y1 to fit within the page size
    
    # Crop the table area
    cropped_page = page.within_bbox(table_bbox)
    
    # Save or display the cropped table section
    cropped_image = cropped_page.to_image()
    cropped_image.save("table_image.png")
